In [ ]:
import os
import tensorflow as tf

IMAGE_SIZE = 128
BATCH_SIZE = 16
BUFFER_SIZE = 1000

def load_image(image_path, image_size=IMAGE_SIZE):
    """Load an image from the disk, resize and normalize."""
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=1)  # Decoding PNG (for glyphs)
    image = tf.image.resize(image, [image_size, image_size])
    image = (image / 127.5) - 1.0  # Normalize to [-1, 1]
    return image

def load_image_pair(conditonal_path, derge_path):
    """Load and return a pair of conditonal and Derge images."""
    conditonal_image = load_image(conditonal_path)
    derge_image = load_image(derge_path)
    return conditonal_image, derge_image

def prepare_dataset(conditonal_dir, derge_dir):
    """Create TensorFlow dataset with paired images from both directories."""
    # Get list of image file names
    conditonal_image_paths = sorted([os.path.join(conditonal_dir, fname) for fname in os.listdir(conditonal_dir)])
    derge_image_paths = sorted([os.path.join(derge_dir, fname) for fname in os.listdir(derge_dir)])
    
    # Create a dataset from the file paths
    dataset = tf.data.Dataset.from_tensor_slices((conditonal_image_paths, derge_image_paths))
    
    # Load images in parallel, shuffle, and batch them
    dataset = dataset.map(lambda x, y: load_image_pair(x, y), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    
    return dataset

# Paths to the directories containing conditonal and Derge glyphs
conditonal_dir = "../data/condition_images"
derge_dir = "../data/derge_glyph_images"

# Prepare the dataset
dataset = prepare_dataset(conditonal_dir, derge_dir)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

def build_generator():
    """Builds the generator model to map a Monlam glyph image to a Derge glyph image."""
    
    # Input: Monlam glyph image
    inputs = layers.Input(shape=(128, 128, 1))
    
    # Encoder
    down1 = layers.Conv2D(64, (4, 4), strides=2, padding='same')(inputs)
    down1 = layers.LeakyReLU()(down1)
    
    down2 = layers.Conv2D(128, (4, 4), strides=2, padding='same')(down1)
    down2 = layers.BatchNormalization()(down2)
    down2 = layers.LeakyReLU()(down2)
    
    down3 = layers.Conv2D(256, (4, 4), strides=2, padding='same')(down2)
    down3 = layers.BatchNormalization()(down3)
    down3 = layers.LeakyReLU()(down3)
    
    down4 = layers.Conv2D(512, (4, 4), strides=2, padding='same')(down3)
    down4 = layers.BatchNormalization()(down4)
    down4 = layers.LeakyReLU()(down4)
    
    # Bottleneck
    bottleneck = layers.Conv2D(512, (4, 4), strides=2, padding='same')(down4)
    bottleneck = layers.ReLU()(bottleneck)
    
    # Decoder (Upsampling)
    up1 = layers.Conv2DTranspose(512, (4, 4), strides=2, padding='same')(bottleneck)
    up1 = layers.BatchNormalization()(up1)
    up1 = layers.ReLU()(up1)
    up1 = layers.Concatenate()([up1, down4])
    
    up2 = layers.Conv2DTranspose(256, (4, 4), strides=2, padding='same')(up1)
    up2 = layers.BatchNormalization()(up2)
    up2 = layers.ReLU()(up2)
    up2 = layers.Concatenate()([up2, down3])
    
    up3 = layers.Conv2DTranspose(128, (4, 4), strides=2, padding='same')(up2)
    up3 = layers.BatchNormalization()(up3)
    up3 = layers.ReLU()(up3)
    up3 = layers.Concatenate()([up3, down2])
    
    up4 = layers.Conv2DTranspose(64, (4, 4), strides=2, padding='same')(up3)
    up4 = layers.BatchNormalization()(up4)
    up4 = layers.ReLU()(up4)
    up4 = layers.Concatenate()([up4, down1])
    
    # Output layer (Derge glyph image)
    outputs = layers.Conv2DTranspose(1, (4, 4), strides=2, padding='same', activation='tanh')(up4)
    
    return tf.keras.Model(inputs, outputs)

# Build the generator
generator = build_generator()
generator.summary()


In [ ]:
def build_discriminator():
    """Builds the discriminator model to classify real vs fake Derge glyph images."""
    
    # Input: Monlam glyph image
    monlam_input = layers.Input(shape=(128, 128, 1))
    
    # Input: Derge glyph image (either real or generated)
    derge_input = layers.Input(shape=(128, 128, 1))
    
    # Combine the condition (Monlam) and the target (Derge) glyph images
    combined_input = layers.Concatenate()([monlam_input, derge_input])
    
    # Discriminator network (PatchGAN)
    down1 = layers.Conv2D(64, (4, 4), strides=2, padding='same')(combined_input)
    down1 = layers.LeakyReLU()(down1)
    
    down2 = layers.Conv2D(128, (4, 4), strides=2, padding='same')(down1)
    down2 = layers.BatchNormalization()(down2)
    down2 = layers.LeakyReLU()(down2)
    
    down3 = layers.Conv2D(256, (4, 4), strides=2, padding='same')(down2)
    down3 = layers.BatchNormalization()(down3)
    down3 = layers.LeakyReLU()(down3)
    
    down4 = layers.Conv2D(512, (4, 4), strides=2, padding='same')(down3)
    down4 = layers.BatchNormalization()(down4)
    down4 = layers.LeakyReLU()(down4)
    
    # Output layer (real/fake classification)
    outputs = layers.Conv2D(1, (4, 4), strides=1, padding='same')(down4)
    
    return tf.keras.Model([monlam_input, derge_input], outputs)

# Build the discriminator
discriminator = build_discriminator()
discriminator.summary()


In [ ]:
import tensorflow as tf

# Compile the cGAN model using the generator and discriminator
def compile_cgan(generator, discriminator, lambda_l1=100):
    # Loss functions
    binary_cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    
    def generator_loss(disc_generated_output, gen_output, target):
        adv_loss = binary_cross_entropy(tf.ones_like(disc_generated_output), disc_generated_output)
        l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
        total_gen_loss = adv_loss + (lambda_l1 * l1_loss)
        return total_gen_loss
    
    def discriminator_loss(disc_real_output, disc_generated_output):
        real_loss = binary_cross_entropy(tf.ones_like(disc_real_output), disc_real_output)
        fake_loss = binary_cross_entropy(tf.zeros_like(disc_generated_output), disc_generated_output)
        total_disc_loss = (real_loss + fake_loss) / 2
        return total_disc_loss
    
    # Optimizers
    generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    
    # Compile the models
    generator.compile(optimizer=generator_optimizer, loss=generator_loss)
    discriminator.compile(optimizer=discriminator_optimizer, loss=discriminator_loss)

# Initialize and compile the model
generator = build_generator()
discriminator = build_discriminator()
compile_cgan(generator, discriminator)

# Callback for saving model weights during training
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

# Function to train the model using tf.keras.Model.fit()
def train_cgan(generator, discriminator, dataset, epochs=100):
    for epoch in range(epochs):
        print(f'Starting epoch {epoch+1}/{epochs}')
        
        for step, (monlam_images, derge_images) in enumerate(dataset):
            # Train the generator and discriminator
            generated_images = generator(monlam_images, training=True)
            
            # Real and fake outputs for discriminator
            real_output = discriminator([monlam_images, derge_images], training=True)
            fake_output = discriminator([monlam_images, generated_images], training=True)
            
            # Compute losses
            gen_loss = generator_loss(fake_output, generated_images, derge_images)
            disc_loss = discriminator_loss(real_output, fake_output)
            
            # Print progress
            if step % 100 == 0:
                print(f'Step {step}, Gen Loss: {gen_loss.numpy()}, Disc Loss: {disc_loss.numpy()}')

# Training the cGAN
train_cgan(generator, discriminator, dataset, epochs=10)
